In [0]:
def prime_factors(n):
    i = 2
    factors = []
    while i * i <= n:
        if n % i:
            i += 1
        else:
            n //= i
            factors.append(i)
    if n > 1:
        factors.append(n)
    return factors


def indices(G,N):
    """takes a group G and a subgroup N and computes list of indices 'n' such that there exists K < G with G/N x G/K = n G/e as G-sets"""

    list_of_conj_class_subs = gap.List(gap.ConjugacyClassesSubgroups(G))

    trivial_intersection_with_N = []

    #This for loop calculates all subgroups with trivial intersection with N < G and adds them to the list defined above.
    #Note that since everything is up to conjugation we throw away a subgroup if any of it's conjugates have non trivial intersection.
    for conj_class in list_of_conj_class_subs:
        throw_away = False
        for ele in gap.List(conj_class):
            if gap.Order(gap.Intersection(ele,N)) == 1:
                pass
            else:
                throw_away = True
                break
        if throw_away == False:
            representative = gap.AsList(conj_class)[1]
            trivial_intersection_with_N.append(representative)

    #we now compute indices using above found subgroups. This is given by |G|/|N||K| for each K with trivial intersection with N.
    indices = []
    for H in trivial_intersection_with_N:
        multiple = int(gap.Order(G))/(int(gap.Order(N))*int(gap.Order(H)))
        if multiple.is_integer():
            indices.append(int(multiple))
    return indices


def test_for_all_non_nilpotent_groups(starting_n = 1,print_order_updates = False, print_k_value_updates = False):
    n = starting_n
    while n > 0:
        k = 1

        if print_order_updates == True:
            if n % 1 == 0:
                print("")
                print(f"Non nilpotent groups of order < {n} have been checked")

        primes_of_n = list(set(prime_factors(n)))

        while k > 0:
            if print_k_value_updates == True:
                if k % 10 == 0:
                    print(f"Order is {n}, groups with k index < {k} have been checked")

            if len(primes_of_n) == 1:
                #This occurs if n = p^k for some prime p, and k >0. i.e. n is a p-group. Then the conjecture says nothing so skip it
                break

            try:
                G = gap.SmallGroup(n,k)
            except RuntimeError:
                #If there's a runtime error then k is too big, therefore we break and move onto the next n value.
                break

            if bool(gap.IsNilpotent(G)) == False:

                for N in gap.List(gap.NormalSubgroups(G)):

                    if min(indices(G,N)) != 1: #i.e. the sequence is not split.

                        #print(f"Order of N is {gap.Order(N)}")
                        #print(f"list of indices is {indices(G,N)}")
                        value1 = True if gcd(indices(G,N)) == 1 else False

                        list_of_mins_for_Sylows = []
                        for p in primes_of_n:
                            Gp = gap.SylowSubgroup(G,p) #Note: only need to test for one sylow p-subgroup for each p.
                            Np = gap.Intersection(Gp,N)
                            list_of_mins_for_Sylows.append(min(indices(Gp,Np)))

                        #print(list_of_mins_for_Sylows)
                        value2 = True if list_of_mins_for_Sylows.count(1) == len(list_of_mins_for_Sylows) else False

                        #print(f"value1 is {value1} and value2 is {value2}")
                        if value1 == True:
                            print("Counterexample to conjecture 1 has been found")
                            print(f"G is SmallGroup({n},{k})")
                            print(f"N is {gap.IdGroup(N)}")
                            H = gap.FactorGroup(G,N)
                            print(f"H is {gap.IdGroup(H)}")

                        if value2 == True:
                            print("Counterexample to conjecture 2 has been found")
                            print(f"G is SmallGroup({n},{k})")
                            print(f"N is {gap.IdGroup(N)}")
                            H = gap.FactorGroup(G,N)
                            print(f"H is {gap.IdGroup(H)}")

                        if value1 != value2:
                            print("***************************************************************")
                            print("Counterexample found to conjecture 1+2!")
                            print("A print statement here is very bad news!")
                            print(f"G is SmallGroup({n},{k})")
                            print(f"N is {gap.IdGroup(N)}")
                            print("***************************************************************")
            k +=1
        n +=1

#test_for_all_non_nilpotent_groups()